
# **Gurobi 5: Programación de Producción con Variables Binarias e Inventarios**  

## **Introducción**  
En esta práctica, aprenderás a modelar un problema de **programación de producción con decisiones binarias y balance de inventarios**.  
El objetivo es decidir **cuándo y cuánto producir** para **minimizar costos** y cumplir con la demanda.  

---  
    


## **Descripción del Problema: Planificación de Producción en una Fábrica de Ropa** 👕👖  

Una fábrica produce **Camisas y Pantalones** durante **4 semanas**.  
Cada producto tiene un **costo variable de producción** y un **costo fijo** si se decide producir en un periodo.  

📌 **Decisiones:**  
- Cuántas unidades de cada producto fabricar en cada semana (**variable continua** $ x_{i,t} $).  
- Si se activa la producción en una semana determinada (**variable binaria** $ y_{i,t} $).  

📌 **Costos:**  
- **Costo variable:** Depende del número de unidades producidas.  
- **Costo fijo:** Se aplica si se decide producir en una semana (activación de línea de producción).  

📌 **Inventario y demanda:**  
- La producción de una semana debe cubrir la demanda y permitir mantener un inventario mínimo.  
- La fábrica tiene un **inventario inicial** y un **costo de almacenamiento**.  
- Se debe cumplir la demanda al final del horizonte de planeación.  

📌 **Parámetros del problema:**  

| Producto  | Costo Variable (\$) | Costo Fijo (\$) | Capacidad Máx. Producción (unidades/semana) | Inventario Inicial | Costo de Almacenamiento (\$/unidad) |
|-----------|----------------|--------------|------------------------------------|-----------------|-------------------------------|
| Camisas   | 10             | 500          | 200                                | 50              | 2                             |
| Pantalones| 15             | 800          | 150                                | 30              | 3                             |  

📌 **Demanda esperada:**  

| Semana | Camisas | Pantalones |
|--------|--------|-----------|
| 1      | 80     | 60        |
| 2      | 100    | 80        |
| 3      | 90     | 70        |
| 4      | 110    | 90        |

📌 **Balance de Inventario:**  
El inventario sigue la ecuación:  

$$
I_{i,t} = I_{i,t-1} + x_{i,t} - D_{i,t}
$$  

Donde:  
- $ I_{i,t} $ es el inventario del producto $ i $ en el periodo $ t $.  
- $ x_{i,t} $ es la cantidad producida.  
- $ D_{i,t} $ es la demanda en el periodo $ t $.  

📌 **Restricciones:**  
1. **Balance de inventario**: Inventario actual = Inventario anterior + Producción - Demanda.  
2. **Capacidad de producción**: No se puede producir más de la capacidad máxima por periodo.  
3. **Producción activada**: Si se produce, se activa el costo fijo.  
4. **No negatividad**: Inventario y producción deben ser mayores o iguales a 0.  

---  
    

## **Paso 1: Definir Parámetros del Modelo**

In [ ]:
# Descargar e instalar Gurobi desde la fuente oficial
!add-apt-repository -y ppa:deadsnakes/ppa
!apt-get update
!apt-get install -y python3.10
!pip install -i https://pypi.gurobi.com gurobipy

In [ ]:
!pip install gurobipy

# Importar Gurobi
import gurobipy as gp
from gurobipy import GRB

In [ ]:

# Conjuntos
productos = ["Camisas", "Pantalones"]
semanas = [1, 2, 3, 4]

# Parámetros
costo_variable = {"Camisas": 10, "Pantalones": 15}
costo_fijo = {"Camisas": 500, "Pantalones": 800}
capacidad_max = {"Camisas": 200, "Pantalones": 150}
inventario_inicial = {"Camisas": 50, "Pantalones": 30}
costo_almacenamiento = {"Camisas": 2, "Pantalones": 3}
demanda = {
    1: {"Camisas": 80, "Pantalones": 60},
    2: {"Camisas": 100, "Pantalones": 80},
    3: {"Camisas": 90, "Pantalones": 70},
    4: {"Camisas": 110, "Pantalones": 90}
}


## **Paso 2: Definir el Modelo de Optimización en Gurobi**

In [ ]:

# Crear el modelo
modelo = gp.Model("Plan_Produccion")

# Variables de decisión
x = modelo.addVars(productos, semanas, vtype=GRB.CONTINUOUS, lb=0, name="Produccion")
y = modelo.addVars(productos, semanas, vtype=GRB.BINARY, name="Activacion")
inventario = modelo.addVars(productos, semanas, vtype=GRB.CONTINUOUS, lb=0, name="Inventario")

# Función Objetivo: Minimizar costos totales (producción + fijos + almacenamiento)
modelo.setObjective(
    gp.quicksum(costo_variable[i] * x[i, t] for i in productos for t in semanas) +
    gp.quicksum(costo_fijo[i] * y[i, t] for i in productos for t in semanas) +
    gp.quicksum(costo_almacenamiento[i] * inventario[i, t] for i in productos for t in semanas),
    GRB.MINIMIZE
)

# Restricciones de balance de inventario
for i in productos:
    for t in semanas:
        if t == 1:
            modelo.addConstr(inventario[i, t] == inventario_inicial[i] + x[i, t] - demanda[t][i])
        else:
            modelo.addConstr(inventario[i, t] == inventario[i, t-1] + x[i, t] - demanda[t][i])

# Restricciones de capacidad de producción
for i in productos:
    for t in semanas:
        modelo.addConstr(x[i, t] <= capacidad_max[i] * y[i, t])

# Resolver el modelo
modelo.optimize()


## **Paso 3: Generar Reporte de Producción e Inventarios**

In [ ]:

# Mostrar la solución óptima
if modelo.status == GRB.OPTIMAL:
    print("Solución óptima encontrada:")
    for i in productos:
        for t in semanas:
            print(f"Semana {t} - {i}: Producción = {x[i, t].x:.0f}, Inventario = {inventario[i, t].x:.0f}, Activado = {int(y[i, t].x)}")
    print(f"Costo total mínimo: {modelo.objVal:.2f}")
else:
    print("No se encontró una solución óptima.")
